#Máster en Big Data y Data Science: Ciencia e Ingeniería de Datos
### ASIGNATURA: Indexación, búsqueda y análisis en repositorios multimedia
### PARTE: Multimedia (imagen, video)
### Práctica 1: Introducción al diseño de redes neuronales convolucionales con Pytorch mediante Google Colaboratory

---

Autor: Juan C. SanMiguel (juancarlos.sanmiguel@uam.es), Universidad Autónoma de Madrid

# 3. Definición de la red neuronal convolucional

## Preparación del entorno de trabajo

A continuación tiene un conjunto de instrucciones que instalan el software necesario para esta parte de la práctica.

Recuerde que este código es compatible con Python 3.

In [1]:
!pip3 install torch==1.10.0+cu111 torchvision==0.11.1+cu111 torchaudio==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.10.0+cu111 (from versions: 0.4.1, 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.1.0.post2, 1.2.0, 1.3.0, 1.3.0.post2, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1)
ERROR: No matching distribution found for torch==1.10.0+cu111


## Red Convolucional Neuronal (CNN)

En esta parte vamos a describir los elementos básicos para definir una red neuronal de tipo *feed-forward*. Este tipo de redes toman una serie de datos de entrada (*input*), éstos son procesados por una serie de capas (*layers*) de manera secuencial y finalmente se genera una salida (*output*) relacionada con la tarea a resolver.

En general, para definir la mayoría de redes convolucionales necesitamos conocer los siguientes pasos:

*   Definir la estructura de la red, que tendrá algunos parámetros entrenables (i.e. *weights*).
*   Definir la secuencia de procesado de los datos para obtener una salida (i.e. *forward pass*).
*   Calcular la función de pérdidas que indique la precisión de la salida con respecto a nuestras anotaciones (i.e. *loss function*).
*   Calcular los gradientes de retropropagación ejecutando en modo inverso la red (i.e. *backward propagation*)
*   Actualización de los parámetros de la red acorde a los gradientes anteriores

En esta parte, vamos a tomar como ejemplo la red LENET http://yann.lecun.com/exdb/lenet/ cuya estructura se visualiza a continuacion:


![alt text](http://pytorch.org/tutorials/_images/mnist.png)

### Capas de una red (layers)
Para definir las capas una red, Pytorch hace uso del paquete ``torch.nn``.

Primeramente podemos definir *capas convolucionales 2D* mediante la función ``nn.Conv2d`` que tiene los principales argumentos:
*   **in_channels**: valor para la tercera dimension del tensor de entrada (e.g. 3 para imágenes RGB, 1 para imágenes en Gris). 
*   **out_channels**: número de mapas de salida (i.e. número de convoluciones o *kernels* que aplicamos sobre los datos de entrada).
*   **kernel_size**: tamaño del *kernel* aplicado (tamaño x tamaño)
*   **stride**: desplazamiento de la aplicación del operador de convolución
*   **padding**: tipo de padding applicado

In [2]:
import torch.nn as nn

conv1 = nn.Conv2d(1, out_channels=6, kernel_size=5, stride=1, padding=0)

print(conv1)

Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))


Posteriormente tenemos *capas con conexión completa* (*fully connected*) mediante la función ``nn.Linear``. Como comparación con la capa convolucional, en esta se asume que todas los datos de entrada están conectados a todos los datos de salida, hecho que aumenta significativamente el número de parámetros. La función tiene los principales argumentos:
*   **in_features**: numero de unidades de entrada. 
*   **out_features**: numero de unidades de entrada

In [3]:
import torch.nn as nn

fc1 = nn.Linear(in_features=240, out_features=120)

print(fc1)

Linear(in_features=240, out_features=120, bias=True)


También exite una etapa dedicada a reducir la dimensionalidad de los datos, cuya nomenclatura es ``nn.MaxPool2d``. Tiene los siguientes argumentos de interés:

*   **kernel_size**: tamaño del *kernel* aplicado (tamaño x tamaño)
*   **stride**: desplazamiento de la aplicación del operador de convolución
*   **padding**: tipo de padding applicado

Es importante resaltar el efecto de esta etapa. Por ejemplo con *kernel_size=2* y *stride=2*, estaremos reduciendo la dimensionalidad de la imagen por dos.


In [4]:
import torch.nn as nn

pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

print(pool1)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


Por último se destaca la operación de activación, implementada mayoritariamente mediante funciones ``ReLU``. En el siguiente código se encuentra comentada debido a que, para ser efectiva, necesita datos 'x' a procesar. Estos datos 'x' se corresponderán con tensor, que no utilizan/cargan en el siguiente bloque de código.

In [5]:
import torch.nn as nn
import torch.nn.functional as F

fc1 = nn.Linear(in_features=240, out_features=120)
#x = F.relu(fc1(x)) #fc1(x) corresponde con aplicar la capa fc1 a unos datos 'x'

print(fc1)

Linear(in_features=240, out_features=120, bias=True)


### Definición red completa
Tras las definiciones anteriores, estamos en condición de mostrar un esquema de la red LENET. 

Una definición básica requiere (al menos) implementar dos funciones:


*   **__init__(self)**: indica el tipo de capas existentes en la red
*   **forward**: indica el flujo de datos, es decir, la secuencia de ejecución cuando llegan nuevos datos a la entrada.



In [6]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        # Convolutional layer 1
        # 1-channel input image, 6 output channels, 5x5 square convolution 
        self.conv1 = nn.Conv2d(1, out_channels=6, kernel_size=5, stride=1, padding=0)
        
        # Max pooling over a (2, 2) window
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Convolutional layer 2
        # 6-channel input data, 16 output channels, 5x5 square convolution 
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5) 
        
        # Fully connected neural network layers 
        # implement an affine operation: y = Wx + b
        # where x-input
        #       y-output
        #       W-weights
        #       b-bias
        self.fc1 = nn.Linear(16 * 5 * 5, 120) #fully connected layer 1
        self.fc2 = nn.Linear(120, 84)         #fully connected layer 2
        self.fc3 = nn.Linear(84, 10)          #fully connected layer 3
        
        #weight initializacion
        #...

    def forward(self, x):
      
        #layer 1: conv + ReLU + pooling
        x = self.pool(F.relu(self.conv1(x)))
        #x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # definicion alternativa

        #layer 2: conv + ReLU + pooling
        x = self.pool(F.relu(self.conv2(x)))
        #x = F.max_pool2d(F.relu(self.conv2(x)), 2) #  definicion alternativa

        #flatten data: convert 2D data into a 1D column vector
        x = x.view(-1, 16 * 5 * 5)
        
        #layer 3: fully connected
        x = F.relu(self.fc1(x))
        
        #layer 4: fully connected
        x = F.relu(self.fc2(x))
        
        #layer 5: fully connected
        x = self.fc3(x)
        return x

NOTA1: observe que *out_channels* de la capa ``conv1`` debe coincidir con la variable *in_channels* de la capa ``conv2``.

NOTA2: similarmente, la salida de la capa ``fc1`` coincide con la entrada de la capa ``fc2``, cuya salida tambien coincide con la entrada de la capa ``fc3``

NOTA3: observe que la salida de la capa ``fc3`` es 10, el número de clases del problema de clasificación que se quiere resolver con la red LENET


Por último, para utilizar esta red debemos crear un objeto de ella:

In [7]:
net = Net()

Adicionalmente podemos las capas que componen la red creada

In [8]:
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Y también podemos ver los parámetros que se pueden aprender mediante entrenamiento.

In [9]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
print(params[2].size())  # conv2's .weight
print(params[4].size())  # fc1's .weight
print(params[6].size())  # fc2's .weight
print(params[8].size())  # fc3's .weight

10
torch.Size([6, 1, 5, 5])
torch.Size([16, 6, 5, 5])
torch.Size([120, 400])
torch.Size([84, 120])
torch.Size([10, 84])


### Ejecución (forward pass)

La ejecución de una red es sencilla:

In [10]:
from torch.autograd import Variable

#create random data with 32x32 dimenssions
input = Variable(torch.randn(1, 1, 32, 32))

#process the data with the network
out = net(input)

#visualize network output
print(out)

tensor([[-0.1313, -0.0510, -0.0849,  0.2580, -0.0406, -0.0416,  0.0711, -0.0018,
         -0.0108,  0.0258]], grad_fn=<AddmmBackward0>)


Cuando deba ejecutar su red, considere los siguientes puntos:


*   Tamaño de entrada esperado: la red que acabamos de definir (Lenet) procesa imágenes de tamaño 32x32. Es decir, si quiere utilizar otro dataset deberá redimensionar las imágenes de entrada.
*   *torch.nn* solamente procesa mini-batches de datos/imágenes (no imágenes individuales). Por ejemplo la función *nn.Conv2d* toma como entrada un tensor 4D ``nSamples x nChannels x Height x Width``
*   La función **Variable** convierte los datos a procesar (tensores) en estructuras enriquecidas que permiten funcionalidades avanzadas (e.g. historial operaciones).

```
# This is formatted as code
```

 

In [13]:
net.zero_grad()
out.backward(torch.randn(1, 10))

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

### Función de pérdidas (loss function)

Una función de pérdida toma el par de entradas (salida, objetivo) y calcula un valor que calcula qué tan lejos está la salida del objetivo. Existen varias [funciones de pérdida](http://pytorch.org/docs/nn.html#loss-functions) en el paquete *nn*.

Una pérdida simple es: `` nn.MSELoss`` que calcula el error cuadrático medio entre la entrada y el objetivo.
Una pérdida simple es: `` nn.CrossEntropyLoss`` que calcula el error entropía cruzada entre la entrada y el objetivo.

A continuación se muestra un ejemplo de ejecución


In [14]:
#generate fake input/output
output = net(input) 
target = Variable(torch.arange(1, 11))  # a dummy target, for example
target = target.to(torch.float32)

#define criterion for loss function
criterion = nn.MSELoss()

#apply loss function
loss = criterion(output, target)
print(loss)

tensor(38.3632, grad_fn=<MseLossBackward0>)


### Retropropagación (backpropagation)

Para poder calcular el error cometido por la red en cada etapa, debemos ejecutar la red en modo inverso mediante la funcion `` loss.backward() ``. Esta función se calcular a partir de la función `` forward `` que hemos proporcionado en la definición de la red. 

Para retropropagar el error, han de seguirse los siguientes pasos:

In [15]:
# You need to clear the existing gradients though, else gradients will be accumulated to existing gradients
net.zero_grad()     # zeroes the gradient buffers of all parameters

#have a look at conv1's bias gradients before and after the backward.
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0184,  0.0432,  0.0604, -0.0113, -0.1083,  0.0823])


### Actualización de parámetros/pesos (update the weights)

En esta parte procedemos a explicar muy brevement el proceso de actualización de los pesos.

Primeramente necesitamos una herramienta de optimización. La ténica más sencilla y comúnmente utilizada es el "descenso estocástico por gradiente" (Stochastic Gradient
Descent, SGD):

     ``weight = weight - learning_rate * gradient``
     
Que podemos implementar fácilmente en Python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)
        
No obstante, existen otras aproximaciones para el proceso de optimización: SGD, Nesterov-SGD, Adam, RMSProp,... que están contenidas en el paquete ``torch.optim``. El siguiente código muestra un ejemplo de utilización del optimizador.


In [16]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

## Bonus: definición de red con entrada genérica
Como se ha comentado anteriormente, la red Lenet solamente puede trabajar con imágenes de gris y tamaño 32x32. En el siguiente bloque de código se muestra una adaptación de la red para procesar imágenes de mayor tamaño (en el ejemplo 224x224) y con varios canales de color (en el ejemplo 3). Observe que se ha añadido la función  ``_get_conv_output`` para calcular las dimensiones de los datos a la entrada de la capa fc1. 

In [17]:
#Original Lenet network
#https://github.com/kuangliu/pytorch-cifar/tree/master/models

#Possible extensions
#https://discuss.pytorch.org/t/inferring-shape-via-flatten-operator/138/3
#https://stackoverflow.com/questions/42479902/how-view-method-works-for-tensor-in-torch

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  
    #define the structure of the network
    def __init__(self, input_shape=(3, 224, 224),num_outputs=15):
        super(Net, self).__init__()
        # Convolutional layer - 3 input image channel, 6 output channels, 5x5 square convolution 
        self.conv1 = nn.Conv2d(in_channels=input_shape[0], out_channels=6, kernel_size=5, stride=1, padding=0)
        
        # Max pooling over a (2, 2) window
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Convolutional layer - 6 input data channel, 16 output channels, 5x5 square convolution 
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)       

        n_size = self._get_conv_output(input_shape)
        
        # Fully connected neural network layers       
        self.fc1 = nn.Linear(in_features=n_size, out_features=120)
        self.fc2 = nn.Linear(in_features=120,    out_features=84)
        self.fc3 = nn.Linear(in_features=84,     out_features=num_outputs)

    #define how data flows through the network
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        #The view function is meant to reshape the tensor (flatten operator).
        x = x.view( x.size(0),-1)
                
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 
        
    def _get_conv_output(self, shape):
        bs = 1
        x = Variable(torch.rand(bs, *shape))
        x = self.pool(F.relu(self.conv1(x)))
        output_feat = self.pool(F.relu(self.conv2(x)))
               
        n_size = output_feat.data.view(bs, -1).size(1)
        return n_size

A continuación, podemos crear una red para un problema dado:

In [18]:
class_names = ('clase1', 'clase2', 'clase3')

#creamos una red con 
# input = imágenes RGB de tamaño 128x128
# output = tres clases
net = Net(input_shape=(3,128,128), num_outputs=len(class_names))
  
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=13456, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=3, bias=True)
)


Si cambiamos el problema, las dimensiones de la red cambian como puede observarse tras ejecutar ``print``:

In [19]:
      
classes = ('clase1', 'clase2', 'clase3', 'clase4', 'clase5')

#creamos una red con 
# input = imágenes RGB de tamaño 128x128
# output = tres clases
net = Net(input_shape=(3,512,512), num_outputs=len(classes))
  
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=250000, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=5, bias=True)
)
